In [1]:
import pandas as pd
import os

# Folder tempat file CSV disimpan
data_path = r"C:\Users\Delta\Downloads\CCS-master\sample data"

# Baca semua dataset
customers = pd.read_csv(os.path.join(data_path, "customers.csv"))
gasstations = pd.read_csv(os.path.join(data_path, "gasstations.csv"))
products = pd.read_csv(os.path.join(data_path, "products.csv"))
sample = pd.read_csv(os.path.join(data_path, "sample.csv"))

# Tampilkan beberapa baris pertama
print("Customers:")
display(customers.head())
print("Gas Stations:")
display(gasstations.head())
print("Products:")
display(products.head())
print("Sample Transactions:")
display(sample.head())


Customers:


,CustomerID,Segment,Currency
0,10,LAM,EUR
1,100,LAM,EUR
2,1000,KAM,CZK
3,10000,SME,CZK
4,10001,SME,CZK


Gas Stations:


,SiteID,chainID,Country,Segment
0,100,10,CZE,Other
1,1000,1,CZE,Premium
2,1001,8,CZE,Value for money
3,1002,24,CZE,Discount
4,1003,13,CZE,Value for money


Products:


,ProductID,Description
0,1,Rucní zadání
1,10,Mazadla/Tuky
2,100,Membership-fee Consorcio
3,101,Diesel
4,102,Biodiesel


Sample Transactions:


,Date,Time,CustomerID,CardID,GasStationID,ProductID,Amount,Price
0,2012-01-01,00:18:00,41113,645177,363,2,93.75,2038.5750
1,2012-01-01,02:05:00,30766,496967,1083,2,132.10,3002.6920
2,2012-01-01,00:27:00,31576,618868,109,5,21.35,462.9240
3,2012-01-01,06:56:00,3800,598481,5298,322,52.50,47.0239
4,2012-01-01,05:46:00,3493,34405,5163,317,70.00,61.8310


In [2]:
print(customers.columns)
print(gasstations.columns)
print(products.columns)
print(sample.columns)


Index(['CustomerID', 'Segment', 'Currency'], dtype='object')
Index(['SiteID', 'chainID', 'Country', 'Segment'], dtype='object')
Index(['ProductID', 'Description'], dtype='object')
Index(['Date', 'Time', 'CustomerID', 'CardID', 'GasStationID', 'ProductID',
       'Amount', 'Price'],
      dtype='object')


In [3]:
# Gabungkan semua dataset ke dalam satu DataFrame
merged_df = sample.merge(customers, on="CustomerID", how="left") \
                  .merge(gasstations, left_on="GasStationID", right_on="SiteID", how="left") \
                  .merge(products, on="ProductID", how="left")

# Lihat hasilnya
merged_df.head()


,Date,Time,CustomerID,CardID,GasStationID,ProductID,Amount,Price,Segment_x,Currency,SiteID,chainID,Country,Segment_y,Description
0,2012-01-01,00:18:00,41113,645177,363,2,93.75,2038.5750,KAM,CZK,363,4,CZE,Premium,Nafta
1,2012-01-01,02:05:00,30766,496967,1083,2,132.10,3002.6920,SME,CZK,1083,1,CZE,Premium,Nafta
2,2012-01-01,00:27:00,31576,618868,109,5,21.35,462.9240,SME,CZK,109,1,CZE,Premium,Natural
3,2012-01-01,06:56:00,3800,598481,5298,322,52.50,47.0239,SME,EUR,5298,130,SVK,Premium,Nat.Super
4,2012-01-01,05:46:00,3493,34405,5163,317,70.00,61.8310,SME,EUR,5163,77,SVK,Premium,Diesel


In [4]:
# Buat kolom total transaksi
merged_df["TotalValue"] = merged_df["Amount"] * merged_df["Price"]

# Hitung total per CustomerID
top5_customers = (
    merged_df.groupby("CustomerID")["TotalValue"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

print("🏆 Top 5 Customers dengan Nilai Transaksi Tertinggi:")
print(top5_customers)


🏆 Top 5 Customers dengan Nilai Transaksi Tertinggi:
CustomerID
6769     1.159793e+06
15470    4.984810e+05
7196     4.331180e+05
30766    3.966556e+05
17693    3.540904e+05
Name: TotalValue, dtype: float64


In [5]:
top5_gasstations = (
    merged_df.groupby("GasStationID")["TotalValue"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

print("⛽ Top 5 Gas Stations dengan Nilai Transaksi Tertinggi:")
print(top5_gasstations)


⛽ Top 5 Gas Stations dengan Nilai Transaksi Tertinggi:
GasStationID
405     1.159793e+06
2367    4.984810e+05
3333    4.331180e+05
1083    3.966556e+05
3671    3.540904e+05
Name: TotalValue, dtype: float64


In [6]:
top5_products = (
    merged_df.groupby("Description")["TotalValue"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

print("🛢️ Top 5 Produk dengan Nilai Transaksi Tertinggi:")
print(top5_products)


🛢️ Top 5 Produk dengan Nilai Transaksi Tertinggi:
Description
Nafta          6.525202e+06
Natural        1.292227e+06
Diesel         1.199222e+05
Ekol.paliva    1.053422e+05
Natural +      8.229886e+04
Name: TotalValue, dtype: float64


In [9]:
# Pastikan kolom Date dalam format tanggal
merged_df["Date"] = pd.to_datetime(merged_df["Date"], errors='coerce')

# Ambil data hanya tanggal 23-26 (kalau ada di dataset)
for day in [23, 24, 25, 26]:
    subset = merged_df[merged_df["Date"].dt.day == day]
    if not subset.empty:
        print(f"\n📆 Statistik Deskriptif untuk tanggal {day}:")
        print(subset.describe())


In [8]:
# Pastikan kolom Time adalah format waktu
merged_df["Time"] = pd.to_datetime(merged_df["Time"], format='%H:%M:%S', errors='coerce').dt.time

# Ambil jam saja
merged_df["Hour"] = pd.to_datetime(merged_df["Time"], format='%H:%M:%S', errors='coerce').dt.hour

# Hitung jumlah transaksi per hari & jam
transactions_per_hour = merged_df.groupby([merged_df["Date"].dt.date, "Hour"]).size().reset_index(name="Total_Transactions")

# Ambil waktu dengan transaksi terbanyak
best_time = transactions_per_hour.sort_values(by="Total_Transactions", ascending=False).head(1)

print("\n🔥 Waktu terbaik (hari dan jam) dengan transaksi terbanyak:")
print(best_time)



🔥 Waktu terbaik (hari dan jam) dengan transaksi terbanyak:
         Date  Hour  Total_Transactions
8  2012-01-01    10                  23
